In [24]:
import folium
import pandas as pd

lats, lons, names, altitude = [],[],[],[]

df = pd.read_csv("data.csv")

lats = df["Latitude"].tolist()
lons = df["Longitude"].tolist()
is_present = [True if present == "sim" else False for present in df["Já é atendido pela Loggi?"].tolist()]


locations = [[lats[i],lons[i]] for i in range(len(lats))]

folium_map = folium.Map(location=locations[0], zoom_start=12)

for idx,local in enumerate(locations):
    folium.Marker(local,icon=folium.Icon(color='green' if is_present[idx] else 'red')).add_to(folium_map)
folium_map



In [14]:
folium_map